# Book recommendation notebook for Model 3

This notebook contains the recommendation algorithm made for model for the book exchange app.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import books dataset
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/book_recommendations/model_2_data.csv',encoding='latin-1')

In [3]:
df.head()

,user_id,title,author,publisher,isbn,language,no_of_exchanges,rating
0,Z4gCnjTUahoz2qaZmk9GJt,Basics of the U.S. Health Care System: Edition 2,Nancy J. Niles,Jones & Bartlett Learning,9.781284e+12,English,3,2.3
1,PwS6wrs4jTJTRrENQ4ErfH,Financial Accounting Theory,Craig Deegan,McGraw-Hill Education Australia,9.781743e+12,English,7,4.7
2,JysYJh9MJsgB92MH86syxU,Delivering Health Care in America: A Systems A...,Leiyu Shi,Jones & Bartlett Learning,9.781284e+12,English,7,1.5
3,ZNanmaD93chZqkq5JHzM2t,Fraud and Fraud Detection: A Data Analytics Ap...,Sunder Gee,John Wiley & Sons,9.781119e+12,English,1,5.0
4,JGUfL4yHZzQhKSLPcMbc3e,Financial Institutions| Instruments and Market...,Christopher Viney,McGraw-Hill Education Australia,9.781744e+12,English,3,1.7


In [4]:
df.shape

(2151, 8)

In [5]:
# unique user id's in the dataset
df['user_id'].unique()

array(['Z4gCnjTUahoz2qaZmk9GJt', 'PwS6wrs4jTJTRrENQ4ErfH',
       'JysYJh9MJsgB92MH86syxU', ..., 'TxfZrGa5ABCS3AEBgkwHbx',
       'dhJ4XcyfKTtYkwbrGT6QT6', 'Ke22KbhUQHPXTufLJBcSsn'], dtype=object)

In [6]:
len(df['user_id'].unique())

2151

In [7]:
df = df.sample(frac = 1)
df.head()

,user_id,title,author,publisher,isbn,language,no_of_exchanges,rating
702,NUaDdu27ucyXKgS9S9hjeJ,Into the Fire,Gregg Hurwitz,Penguin UK,9.781406e+12,English,5,4.7
520,2KRm64ZTp4t57Ju7Qrx66R,The Travelling Cat Chronicles: The most uplift...,Hiro Arikawa,Random House,9.781474e+12,English,7,4.5
868,NbyaQajPWQdrJj4DHTtCCE,Wonderment In Death,J. D. Robb,Hachette UK,9.780752e+12,English,2,4.7
1902,CKxNoTnz6RE6hF4592R9oS,These Ghosts Are Family: A Novel,Maisy Card,Simon and Schuster,9.781982e+12,English,2,4.8
1738,dZShKdUBMksedqazirTCHM,The Royal Nanny: A Novel,Karen Harper,HarperCollins,9.780062e+12,English,7,3.9


In [8]:
df.dtypes

user_id              object
 title               object
 author              object
 publisher           object
 isbn               float64
 language            object
 no_of_exchanges      int64
 rating             float64
dtype: object

In [9]:
df.isna().sum()

user_id               0
 title                0
 author               4
 publisher            0
 isbn               207
 language             1
 no_of_exchanges      0
 rating               0
dtype: int64

In [10]:
df.dropna(inplace=True)

In [11]:
df.isna().sum()

user_id             0
 title              0
 author             0
 publisher          0
 isbn               0
 language           0
 no_of_exchanges    0
 rating             0
dtype: int64

In [12]:
from scipy.sparse import csr_matrix
# pivot rating into movie features
mat = df.pivot(index=' isbn',columns='user_id',values=' no_of_exchanges').fillna(0)

In [13]:
mat.head()

user_id,22gThuJzsUrfdmEnA6stVe,22z5Cgp2oWgwcfXULdDMyv,237AfG45kE6tKG9KXrhmEh,25M8xKKYVweuPCMxfinYzu,25aAPXXm4cLAgwohtb7Q7F,25wgGugMawBspS8Z8BYHpK,27tCDQZw7Mpyx3RWoS6YX9,27zZfwYsi6m9EDo73XCWNn,29TKi2jiZQcAFAXpfbuC9F,2CdhCconXp93GuFFb4nq6i,2DB5qhhqW6z4KJPpkVPKiq,2Fr4kA38kNTtbMddUjZMbE,2JaepgyTUkpk8TY4PCqfq8,2KJtWmEBxRWUa9Cm6sbQBw,2KQrD6dSF3pWCa7pbENdCg,2KRm64ZTp4t57Ju7Qrx66R,2PYQD4Z32bGdanY6N6XAyv,2PkFbLhXiW3ttY3TwaGbh7,2PtuqH3TcyUjKqxmExBH7b,2QM5SU3hypCzsi9BBNpsTN,2QxRkSakxP36uxyYHDkZM4,2RbcbxzCn7RZ7asR9NsdsZ,2Rh5MibEfR8edvnKyqXUjr,2TdCP2EAqosv7Y2Zono36Y,2TwT8wT5RbdaF9qGLAc55B,2U98DYvvdAAPX7sgsz4Zdj,2UW6EnY4tfh2BKhoaYbxWP,2VsvcQR7WfNy7J8i7HmTL4,2bstVJWsXyrkbLwmoTb29W,2dCQJFTro2Da627U2CViHk,2doXTWzm5ncmM7BaRoz7Qy,2hKQzsyNMJorYa3kdeQYn2,2i9g6kN3fqUJ9yEzuh499E,2jXxzRNeLZGoD3g6G5BU5g,2jbuUvA7NxdV4ZNZQTqWFt,2ns6EUYic4Nb4BZGN94ULG,2oq5bjSR4xDW67yuhdRBrn,2pFXpMJCTAH9jPd63jix9g,2rM73GEyC9BGmDXjvic3wC,2smTV8t3HB3hRhWh2EsxdQ,...,nmicBono9shVi3aBcyd46W,nnQLLuWGJMpqkPSyRrSfQG,noAtJQouJipjzt8DPdekDe,npPeTZaZVAe7CcyrEDT4sL,npTrKwagnR8XfCPrxx3MHZ,nq8xJRNfYzn6Zzx33BYTBQ,nqSDsiLkcSfsNLLjebz2LD,nuCx2nzw3z8vGjGndxsZRR,nvUdvwHGJVSjipHnTX2D4s,nw5FAcfwxm5Th3jJyFDAkj,nwDRLW7JYf3LNTQ3s58JG9,nwqCX93dKPS4ZFoEEU54eL,nwy2cM4WXBnwDkRVebkXYE,nxHYuHwbocMYBhauoT3Wp7,nxgr27UozQyvyoZdDUTfbZ,nxqymP6iPqUP7E5m4dTR3q,o2McYrqGM7DL6EJTcUXSuc,o3UDoMFD87RMLSpdYKpPJV,o4SmZBLPWtqsZdYgpTVuYw,o6vvyvoMF2ABqwiZWnFPRM,o9iYZm5cd4KfXjwFdwW9MU,oC9XoikDeGmDopviQWQ2eG,oCYQoGDX8xJb9DGRTFJkdp,oH2rWdFEB5ph7y6Qss7hUA,oJRBzhruqz3nn2kpKXG9Xf,oK9TseNuCRDorXdysfDETa,oLkYBkdSATbpPQGAdLbZfT,oM538McsA4JkXWsKChUa4G,oMBf6ArLXDRJMeN5wfJcJs,oMkmqsyReQGwFyj2newTqt,oNmCQ9XvKwbtYgUsPGt9Yi,oP7JHFp4nb6bzyTbSMD52y,oPo8FadNFVE7NRw8vpuZyz,oPyoqoKG84pZ2csYdCNWYF,oQruRduMi4jSN52DTQNek5,oSGamAutzQYdKguVthSBvs,oWZNMmZ8mPuHwpKmrDiVw5,oXQRFWkeJfYuaYBfWB4pnT,oXtcafmg7M8EHnPtg8HXjC,oZ6gpQWsByz8gHGgS38s3C
isbn,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
mat.shape

(1544, 1939)

In [15]:
sparse_mat = csr_matrix(mat.values)

In [16]:
book_map = df[[' isbn', ' title']]

In [17]:
book_map.head()

,isbn,title
702,9.781406e+12,Into the Fire
520,9.781474e+12,The Travelling Cat Chronicles: The most uplift...
868,9.780752e+12,Wonderment In Death
1902,9.781982e+12,These Ghosts Are Family: A Novel
1738,9.780062e+12,The Royal Nanny: A Novel


In [18]:
book_map

,isbn,title
702,9.781406e+12,Into the Fire
520,9.781474e+12,The Travelling Cat Chronicles: The most uplift...
868,9.780752e+12,Wonderment In Death
1902,9.781982e+12,These Ghosts Are Family: A Novel
1738,9.780062e+12,The Royal Nanny: A Novel
...,...,...
89,9.780062e+12,Wine Grapes: A Complete Guide to 1|368 Vine Va...
1523,9.781474e+12,Lock Every Door
1721,9.789387e+12,MANDESHI MANASA
1097,9.781452e+12,Petals on the Wind


In [19]:
book_map.values

array([[9781405928571.0, 'Into the Fire'],
       [9781473541573.0,
        'The Travelling Cat Chronicles: The most uplifting book of the year'],
       [9780751554106.0, 'Wonderment In Death'],
       ...,
       [9789386745675.0, 'MANDESHI MANASA'],
       [9781451636956.0, 'Petals on the Wind'],
       [9780297860242.0,
        'Sharp Objects: A major HBO & Sky Atlantic Limited Series starring Amy Adams| from the director of BIG LITTLE LIES| Jean-Marc Vallée']],
      dtype=object)

In [20]:
hash_map = {
    movie: i for i,movie in enumerate(list(book_map.set_index(' isbn').loc[mat.index][' title']))
}

In [21]:
hash_map

{'Half of a Yellow Sun': 3,
 'PS| I Love You: The emotional| heartbreaking romantic fiction book from the number one best selling author of Postscript': 4,
 'Love Rosie': 7,
 'Thanks for the Memories': 8,
 'Lords of the Bow (Conqueror| Book 2)': 9,
 'HMS Ulysses': 11,
 'San Andreas': 12,
 'Temeraire (The Temeraire Series| Book 1)': 14,
 'The Fellowship of the Ring (The Lord of the Rings| Book 1)': 15,
 'The Return of the King (The Lord of the Rings| Book 3)': 16,
 'Unfinished Tales': 17,
 'The Children of Húrin': 18,
 'The Lord of the Rings: The classic fantasy masterpiece': 20,
 'Mere Christianity': 22,
 'Sharpe\x92s Tiger: The Siege of Seringapatam| 1799 (The Sharpe Series| Book 1)': 23,
 'The Last Kingdom (The Last Kingdom Series| Book 1)': 24,
 'Remarkable Creatures': 25,
 'A Foreign Country (Thomas Kell Spy Thriller| Book 1)': 26,
 'The Demon King (The Seven Realms Series| Book 1)': 27,
 'The Homecoming of Samuel Lake': 28,
 'Balthazar Jones and the Tower of London Zoo': 29,
 'Ame

In [22]:
!pip install fuzzywuzzy

In [23]:
# function for searching books in database using keywords
from fuzzywuzzy import fuzz 
def matching(book_map, fav_book):
  """
  returns the closest match via fuzzy ratio
  If no match found, returns None

  Parameters
  --------
  book_map : dataframe of book titles along with their isbn
  fav_book : name of the favourite book

  Return
  -------
  index of closest match
  """
  match = []
  # get match
  for title,isbn in book_map.items():
    ratio = fuzz.ratio(title.lower(),fav_book.lower())
    if ratio >=60:
      match.append((title,isbn, ratio))
    # sort
  match = sorted(match,key = lambda x:x[2])[::-1]
  if not match:
    print("No match found!!!")
  else:
    print("Found possible matches in our database: {0}".format([x[0] for x in match]))
    return match[0][1]

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [24]:
matching(hash_map,"Requiem for Dream")

Found possible matches in our database: ['Requiem for a Dream: A Novel']


1245

In [25]:
# function for making recommendations
import time
from sklearn.neighbors import NearestNeighbors
def recommend(fav_book,n_recommendations,book_map,data):
  """
  Returns the top n recommendations based on the keyword entered

  Parameters
  ---------
  fav_book : Keyword for favourite book
  n_recommendations : Number of recommendations to be made
  book_map : book database with title and isbn
  data : csr matrix of user_id, ratings and isbn

  Return
  --------
  list of top n book recommendations
  """
  # instantiating model
  model = NearestNeighbors(algorithm ='brute',metric='cosine')
  # fitting the model
  model.fit(data)
  # get input book isbn
  book_isbn = matching(book_map, fav_book)
  # inference
  print("Recommendation system starting to make inference......")
  t0 = time.time()
  distances, isbns = model.kneighbors(
      data[book_isbn],
      n_neighbors=n_recommendations+1
  )
  recommends = sorted(list(zip(isbns.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x : x[1])[:0:-1]
  print(f"Time Taken for recommendation : {time.time()-t0}")
  return recommends 

In [26]:
# function to show movie names after recommedations
def make_recommendations(fav_book, n_recommendations, mat, hash_map):
  """
  makes n recommendations based on fav books
  Parameters
  ----------
  fav_book : favourite book
  n_recommendation : number of recommedations to be made
  Return
  ----------
  None, print recommendations
  """
  raw_recommendations = recommend(fav_book,n_recommendations=10,book_map=hash_map, data=mat)
  reverse_hashmap = {v : k for k,v in hash_map.items}
  print('Recommendations for {}:'.format(fav_book))
  for i,(isbn,dist) in enumerate(raw_recommendations):
    print('{0}: {1}, with distance of {2}'.format(i+1,reverse_hashmap[isbn],dist))

In [27]:
book_map

,isbn,title
702,9.781406e+12,Into the Fire
520,9.781474e+12,The Travelling Cat Chronicles: The most uplift...
868,9.780752e+12,Wonderment In Death
1902,9.781982e+12,These Ghosts Are Family: A Novel
1738,9.780062e+12,The Royal Nanny: A Novel
...,...,...
89,9.780062e+12,Wine Grapes: A Complete Guide to 1|368 Vine Va...
1523,9.781474e+12,Lock Every Door
1721,9.789387e+12,MANDESHI MANASA
1097,9.781452e+12,Petals on the Wind


In [28]:
book = "The World"
recommend(book,n_recommendations=10,book_map=hash_map,data=sparse_mat)

Found possible matches in our database: ['The Lost World', 'The Wren', 'The World in Half', 'The New Girl', 'The Wolf Gift', 'The Goldfinch', 'The Cold Dish', 'Fatherland', 'The Terror', 'The Savior', "The Doctor's Lady", 'The War of the Worlds']
Recommendation system starting to make inference......
Time Taken for recommendation : 0.00875997543334961


[(1027, 1.0),
 (1033, 1.0),
 (1025, 1.0),
 (1034, 1.0),
 (1035, 1.0),
 (1032, 1.0),
 (1030, 1.0),
 (1029, 1.0),
 (1028, 1.0),
 (1031, 1.0)]

In [29]:
book_map[book_map[' isbn']==9781407097626]

,isbn,title
852,9.781407e+12,The Lost World


In [30]:
print(sparse_mat[1])

  (0, 862)	5.0


In [31]:
hash_map

{'Half of a Yellow Sun': 3,
 'PS| I Love You: The emotional| heartbreaking romantic fiction book from the number one best selling author of Postscript': 4,
 'Love Rosie': 7,
 'Thanks for the Memories': 8,
 'Lords of the Bow (Conqueror| Book 2)': 9,
 'HMS Ulysses': 11,
 'San Andreas': 12,
 'Temeraire (The Temeraire Series| Book 1)': 14,
 'The Fellowship of the Ring (The Lord of the Rings| Book 1)': 15,
 'The Return of the King (The Lord of the Rings| Book 3)': 16,
 'Unfinished Tales': 17,
 'The Children of Húrin': 18,
 'The Lord of the Rings: The classic fantasy masterpiece': 20,
 'Mere Christianity': 22,
 'Sharpe\x92s Tiger: The Siege of Seringapatam| 1799 (The Sharpe Series| Book 1)': 23,
 'The Last Kingdom (The Last Kingdom Series| Book 1)': 24,
 'Remarkable Creatures': 25,
 'A Foreign Country (Thomas Kell Spy Thriller| Book 1)': 26,
 'The Demon King (The Seven Realms Series| Book 1)': 27,
 'The Homecoming of Samuel Lake': 28,
 'Balthazar Jones and the Tower of London Zoo': 29,
 'Ame

In [32]:
recommendations = recommend(book,n_recommendations=10,book_map=hash_map,data=sparse_mat)

Found possible matches in our database: ['The Lost World', 'The Wren', 'The World in Half', 'The New Girl', 'The Wolf Gift', 'The Goldfinch', 'The Cold Dish', 'Fatherland', 'The Terror', 'The Savior', "The Doctor's Lady", 'The War of the Worlds']
Recommendation system starting to make inference......
Time Taken for recommendation : 0.002391815185546875


In [33]:
recommendations


[(1027, 1.0),
 (1033, 1.0),
 (1025, 1.0),
 (1034, 1.0),
 (1035, 1.0),
 (1032, 1.0),
 (1030, 1.0),
 (1029, 1.0),
 (1028, 1.0),
 (1031, 1.0)]

In [34]:
for (i,j) in recommendations:
  for title, isbn in hash_map.items():
    if isbn == i:
      print(title)    

The Noticer: Sometimes| all a person needs is a little perspective
Honolulu: A Novel
Return to Summerhouse
Finding Jack: A Novel
Merry Humbug Christmas: Two Tales of Holiday Romance
One Good Dog: A Novel
His Little Cowgirl: A Wholesome Western Romance
Why They Can't Write: Killing the Five-Paragraph Essay and Other Necessities
Barbarous
All He Ever Needed


In [35]:
book_dict = dict(zip(book_map[' isbn'],book_map[' title']))

In [36]:
# function to get list of recommmendations based on isbn
def recommendations_from_isbn(isbn,data,n_neighbors,book_dict,hash_map):
  """
  This function provides recommendations based on isbn
  
  Parameters
  ---------
  isbn: isbn of the book from which recommendations have to be made
  data: sparse matrix data
  n_neighbors: number of neighbors for knn models
  book_dict: dictionary linking isbn to book name
  hash_map: dictionary linking title to sparse matrix index

  Return
  --------
  recoms: recommendations from the passed isbn
  """
  dummy_model = NearestNeighbors(algorithm='brute',metric='cosine')
  dummy_model.fit(data)
  title = book_dict[isbn]
  r,d = dummy_model.kneighbors(data[hash_map[title]],n_neighbors=11)
  recoms = []
  recommends = sorted(list(zip(d.squeeze().tolist(),r.squeeze().tolist())),key=lambda x : x[1])[:0:-1]
  for (i,j) in recommends:
    for title, isbn in hash_map.items():
      if isbn == i:
        recoms.append(title)
  return recoms

In [37]:
recommendations_from_isbn(9780984835935.0,data=sparse_mat,n_neighbors=10, book_dict=book_dict, hash_map=hash_map)

['The Noticer: Sometimes| all a person needs is a little perspective',
 'One Good Dog: A Novel',
 'Gone with the Wind',
 'Honolulu: A Novel',
 'Finding Jack: A Novel',
 'Fireproof',
 "Why They Can't Write: Killing the Five-Paragraph Essay and Other Necessities",
 'Barbarous',
 'All He Ever Needed',
 'His Little Cowgirl: A Wholesome Western Romance']

In [38]:
# function to get recommended books' isbn
def get_books_isbn(isbn,data,n_neighbors,book_dict,hash_map):
  """
  Function to return isbns of recommended books from the isbn passed

  Parameters
  ---------
  isbn: isbn of the book from which recommendations have to be made
  data: sparse matrix data
  n_neighbors: number of neighbors for knn models
  book_dict: dictionary linking isbn to book name
  hash_map: dictionary linking title to sparse matrix index

  Return
  --------
  isbns: isbns of recommendations from the passed isbn

  """
  recoms = recommendations_from_isbn(isbn,data=data,n_neighbors=n_neighbors,book_dict=book_dict,hash_map=hash_map)
  isbns = []
  for book_name in recoms:
    for isb,title in book_dict.items():
      if title == book_name:
        isbns.append(isb)
  return isbns

In [39]:
get_books_isbn(9780748117352.0,data=sparse_mat,n_neighbors=10, book_dict=book_dict, hash_map=hash_map)

[9781418574567.0,
 9781429994798.0,
 9781416596479.0,
 9781429977739.0,
 9781426894336.0,
 9781418566722.0,
 9781421427119.0,
 9781420147247.0,
 9781429959308.0,
 9781426823442.0]